## SI4463 Class-E Match for Direct Tie Board Configuration @435MHz

These calculations are based on the datasheet AN648

### Parameters & Imports

In [1]:
#Gebaseerd op AN648
import math
import cmath
import numpy
from si_prefix import si_format
from eseries import find_nearest, E12


freq = 436e6 #Hz
TxGain = 20 #dBm
Vdd = 3.3 #V

C0 = 8.2e-12 # can be freely chosen

CLNA = 0.97 * 10**(-12)
RLNA = 480
ReAnt = 50
ImAnt = 0
CShunt =  2.5e-12

Pout = 10**(TxGain/10)*1e-3 #Output power in Watt
WRF = freq * math.pi * 2

#### Step #1 Select a value for  Lchoke pull-up Inductor

Based on the values recommended by the datasheet for a frequency of 433MHz the value should be around 253.9nH. Found by using regression (power => $y = 556030x^{-1.267}$)

In [12]:
Lc = 253.9e-9
print("Lchoke: " + si_format(Lc))

Lchoke: 253.9 n


#### Step #2: Choose/Calculate Values for L0-C0 Series-Resonant Tank

In [13]:
L0 = 1 / ((math.pi * freq * 2)**2 * C0)
L0 = L0 * 1.2 # mistuning for direct tie

#### Step #3: Calculate the Required Value for ZLOAD

In [14]:
angle = math.radians(49.0524)
Zload = (0.2815 / (2 * math.pi * freq * CShunt))* cmath.exp(1j*angle)
print(Zload)

(26.999445848719645+31.11673900457209j)


#### Step # 4: Calculate the Required Value for Voltage-Limiting Resistor RDC

In [21]:
Vdd_Pa = math.sqrt(Pout / (math.pi**2 * 2 * freq * CShunt))
Idd_Pa = math.pi**2 * 2 * freq * CShunt * Vdd_Pa
RDC = (Vdd - Vdd_Pa) / Idd_Pa
print(find_nearest(E12,RDC)) # it will almost always be necessary to use a smaller value of RDC than predicted by theory in order to obtain the desired value of output power. Because of losses 

27.0


#### Step #5: Calculate the Values for Matching Components LX and CM

In [16]:
# Should be done with smith cards and is based on ZAnt and Zload
Cm = 6.7e-12 
Lx = 20.5e-9
L0 = L0 + Lx

#### Step #6: Design a Lowpass Filter

#### Calculating order

In [17]:
a_min = 30
ripple = 0.45
omega_s = 2

Nd = math.acosh(math.sqrt((math.pow(10, (a_min/10))-1)/(math.pow(10,(ripple/10))-1)))/(math.acosh(omega_s))
print(f"orde: {math.ceil(Nd)} (Nd={Nd})")


orde: 4 (Nd=3.9894211469293377)


In [18]:
#based on https://www.changpuak.ch/electronics/chebyshev_lowpass.php

ripple = 0.25 #dB
z = 50 #ohm
n = math.ceil(Nd)
shunt = True # shunt or series
w = WRF/1e6
e2x = math.exp(2*ripple/17.37)
coth = (e2x+1)/(e2x-1)
btn = math.log(coth)/(2*n)
gn = (math.exp(btn)-math.exp(-btn))/2
a = numpy.zeros(n+1)
b = numpy.zeros(n+1)
g = numpy.zeros(n+1)
result = ""

for i in range(1,n+1):
    a[i] = math.sin((2*i-1)*math.pi/(2*n))
    k3 = math.sin(math.pi*i/n)
    b[i] = gn**2 + k3**2

g[1] = 2*a[1]/gn

for i in range(2,n+1):
    g[i] = (4*a[i-1]*a[i])/(b[i-1]*g[i-1])

if shunt:
    for i in range(1,n+1):
        if i % 2: print(f"element {i} orientation : shunt, C = {find_nearest(E12,round(g[i]*1e9/(z*w))/1000)} pF")
        else: print(f"element {i} orientation : series, L = {find_nearest(E12,round(g[i]*1e6*z/w)/1000)} nH")
else: 
    for i in range(1,n+1):
        if i % 2 == 0: print(f"element {i} orientation : shunt, C = {find_nearest(E12,round(g[i]*1e9/(z*w))/1000)} pF")
        else: print(f"element {i} orientation : series, L = {find_nearest(E12,round(g[i]*1e6*z/w)/1000)} nH")

element 1 orientation : shunt, C = 10.0 pF
element 2 orientation : series, L = 22.0 nH
element 3 orientation : shunt, C = 15.0 pF
element 4 orientation : series, L = 15.0 nH


#### RX imput match

In [19]:
LLNA = 1 / (WRF**2 * CLNA)
LR2 = math.sqrt(ReAnt * RLNA) / WRF
LM = (2 * LR2) / (((2 * ImAnt) / (WRF * LR2)) + 1)
LR1 = (LLNA * LM) / (LLNA + LM)
CR2 = 1 / (WRF**2 * LR2)
CR1 = 2 * CR2

In [20]:
print(f"Cm: {si_format(Cm)} ({si_format(find_nearest(E12, Cm))})")
print(f"Lc: {si_format(Lc)} ({si_format(find_nearest(E12, Lc))})")
print(f"C0: {si_format(C0)} ({si_format(find_nearest(E12, C0))})")
print(f"L0: {si_format(L0)} ({si_format(find_nearest(E12, L0))})")
print(f"LR1: {si_format(LR1)} ({si_format(find_nearest(E12, LR1))})")
print(f"LR2: {si_format(LR2)} ({si_format(find_nearest(E12, LR2))})")
print(f"CR1: {si_format(CR1)} ({si_format(find_nearest(E12, CR1))})")
print(f"CR2: {si_format(CR2)} ({si_format(find_nearest(E12, CR2))})")

Cm: 6.7 p (6.8 p)
Lc: 253.9 n (270.0 n)
C0: 8.2 p (8.2 p)
L0: 40.1 n (39.0 n)
LR1: 62.2 n (68.0 n)
LR2: 56.7 n (56.0 n)
CR1: 4.7 p (4.7 p)
CR2: 2.4 p (2.2 p)
